In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time

from torchvision import datasets, transforms

device = torch.device("cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)

train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc2 = nn.Linear(50,50)
        self.fc3 = nn.Linear(50,50)
        self.fc4 = nn.Linear(50,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [3]:
def train_model(model, num_epochs):
    learning_rate = 0.0001
    opt = optim.Adam(params=model.parameters(), lr=learning_rate)
    ce_loss = torch.nn.CrossEntropyLoss()

    for epoch in range(1,num_epochs+1):
        t1 = time.time()

        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)      
            opt.zero_grad()
            out = model(x_batch)
            batch_loss = ce_loss(out, y_batch)
            batch_loss.backward()
            opt.step()

        tot_test, tot_acc = 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))

In [4]:
base_model = nn.Sequential(Net())
base_model = base_model.to(device)
base_model.train()
train_model(base_model, 30)

/home/aman/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1: Accuracy 0.86280 [9.43 seconds]
Epoch 2: Accuracy 0.90360 [11.19 seconds]
Epoch 3: Accuracy 0.91270 [10.79 seconds]
Epoch 4: Accuracy 0.91880 [11.07 seconds]
Epoch 5: Accuracy 0.92410 [10.66 seconds]
Epoch 6: Accuracy 0.92990 [10.97 seconds]
Epoch 7: Accuracy 0.93310 [10.62 seconds]
Epoch 8: Accuracy 0.93580 [11.56 seconds]
Epoch 9: Accuracy 0.93970 [10.62 seconds]
Epoch 10: Accuracy 0.94150 [9.90 seconds]
Epoch 11: Accuracy 0.94430 [10.37 seconds]
Epoch 12: Accuracy 0.94610 [10.85 seconds]
Epoch 13: Accuracy 0.94980 [11.13 seconds]
Epoch 14: Accuracy 0.95010 [10.84 seconds]
Epoch 15: Accuracy 0.95350 [10.52 seconds]
Epoch 16: Accuracy 0.95540 [10.45 seconds]
Epoch 17: Accuracy 0.95700 [10.56 seconds]
Epoch 18: Accuracy 0.95690 [9.90 seconds]
Epoch 19: Accuracy 0.95770 [11.18 seconds]
Epoch 20: Accuracy 0.96040 [11.24 seconds]
Epoch 21: Accuracy 0.96100 [11.49 seconds]
Epoch 22: Accuracy 0.96040 [11.16 seconds]
Epoch 23: Accuracy 0.96250 [10.21 seconds]
Epoch 24: Accuracy 0.96

In [5]:
def get_interval_input(img, delta):
    il = torch.maximum(img - delta, torch.tensor(0))
    iu = torch.minimum(img + delta, torch.tensor(1))
    return il, iu

In [6]:
def get_robust_accuracy(model, delta):
    tot_test, tot_acc = 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        w0,b0,w1,b1,w2,b2,w3,b3 = model.parameters()
        b0, b1, b2, b3 = b0.reshape(-1,1), b1.reshape(-1,1), b2.reshape(-1,1), b3.reshape(-1,1)
        
        il, iu = get_interval_input(x_batch, delta)
        il, iu = il.view((-1,784,1)), iu.view((-1,784,1))
            
        l1l = torch.matmul(torch.maximum(w0, torch.tensor(0)), il) + torch.matmul(torch.minimum(w0, torch.tensor(0)), iu)
        l1u = torch.matmul(torch.maximum(w0, torch.tensor(0)), iu) + torch.matmul(torch.minimum(w0, torch.tensor(0)), il)
        l1l, l1u = torch.maximum(l1l + b0, torch.tensor(0)), torch.maximum(l1u + b0, torch.tensor(0))

        l2l = torch.matmul(torch.maximum(w1, torch.tensor(0)), l1l) + torch.matmul(torch.minimum(w1, torch.tensor(0)), l1u)
        l2u = torch.matmul(torch.maximum(w1, torch.tensor(0)), l1u) + torch.matmul(torch.minimum(w1, torch.tensor(0)), l1l)
        l2l, l2u = torch.maximum(l2l + b1, torch.tensor(0)), torch.maximum(l2u + b1, torch.tensor(0))

        l3l = torch.matmul(torch.maximum(w2, torch.tensor(0)), l2l) + torch.matmul(torch.minimum(w2, torch.tensor(0)), l2u)
        l3u = torch.matmul(torch.maximum(w2, torch.tensor(0)), l2u) + torch.matmul(torch.minimum(w2, torch.tensor(0)), l2l)
        l3l, l3u = torch.maximum(l3l + b2, torch.tensor(0)), torch.maximum(l3u + b2, torch.tensor(0))

        ol = torch.matmul(torch.maximum(w3, torch.tensor(0)), l3l) + torch.matmul(torch.minimum(w3, torch.tensor(0)), l3u)
        ou = torch.matmul(torch.maximum(w3, torch.tensor(0)), l3u) + torch.matmul(torch.minimum(w3, torch.tensor(0)), l3l)
        ol, ou = ol + b3, ou + b3

        for idx, t in enumerate(y_batch):
            robust = True
            for j in range(10):
                if j != t:
                    robust = robust and (ol[idx][t] > ou[idx][j])
            tot_acc += 1 if robust else 0
    
        tot_test += x_batch.size()[0]
    print('Epsilon %.3lf: Robust accuracy %.5lf' % (delta, tot_acc/tot_test))

In [7]:
for i in range(1,11):
    get_robust_accuracy(base_model, 0.01*i)

Epsilon 0.010: Robust accuracy 0.00000
Epsilon 0.020: Robust accuracy 0.00000
Epsilon 0.030: Robust accuracy 0.00000
Epsilon 0.040: Robust accuracy 0.00000
Epsilon 0.050: Robust accuracy 0.00000
Epsilon 0.060: Robust accuracy 0.00000
Epsilon 0.070: Robust accuracy 0.00000
Epsilon 0.080: Robust accuracy 0.00000
Epsilon 0.090: Robust accuracy 0.00000
Epsilon 0.100: Robust accuracy 0.00000


In [8]:
def robust_train_model(model, num_epochs, delta, kappa_min, kappa_max):
    learning_rate = 0.001
    opt = optim.Adam(params=model.parameters(), lr=learning_rate)
    fit_loss = torch.nn.CrossEntropyLoss()
    spec_loss = torch.nn.CrossEntropyLoss()

    for epoch in range(1,num_epochs+1):
        t1 = time.time()
        
        kappa = kappa_max - (kappa_max-kappa_min)*(epoch-1)/(num_epochs-1)
        
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)   
            opt.zero_grad()

            w0,b0,w1,b1,w2,b2,w3,b3 = model.parameters()
            b0, b1, b2, b3 = b0.reshape(-1,1), b1.reshape(-1,1), b2.reshape(-1,1), b3.reshape(-1,1)
            
            il, iu = get_interval_input(x_batch, delta)
            il, iu = il.view((-1,784,1)), iu.view((-1,784,1))
            
            l1l = torch.matmul(torch.maximum(w0, torch.tensor(0)), il) + torch.matmul(torch.minimum(w0, torch.tensor(0)), iu)
            l1u = torch.matmul(torch.maximum(w0, torch.tensor(0)), iu) + torch.matmul(torch.minimum(w0, torch.tensor(0)), il)
            l1l, l1u = torch.maximum(l1l + b0, torch.tensor(0)), torch.maximum(l1u + b0, torch.tensor(0))
            
            l2l = torch.matmul(torch.maximum(w1, torch.tensor(0)), l1l) + torch.matmul(torch.minimum(w1, torch.tensor(0)), l1u)
            l2u = torch.matmul(torch.maximum(w1, torch.tensor(0)), l1u) + torch.matmul(torch.minimum(w1, torch.tensor(0)), l1l)
            l2l, l2u = torch.maximum(l2l + b1, torch.tensor(0)), torch.maximum(l2u + b1, torch.tensor(0))
            
            l3l = torch.matmul(torch.maximum(w2, torch.tensor(0)), l2l) + torch.matmul(torch.minimum(w2, torch.tensor(0)), l2u)
            l3u = torch.matmul(torch.maximum(w2, torch.tensor(0)), l2u) + torch.matmul(torch.minimum(w2, torch.tensor(0)), l2l)
            l3l, l3u = torch.maximum(l3l + b2, torch.tensor(0)), torch.maximum(l3u + b2, torch.tensor(0))
            
            ol = torch.matmul(torch.maximum(w3, torch.tensor(0)), l3l) + torch.matmul(torch.minimum(w3, torch.tensor(0)), l3u)
            ou = torch.matmul(torch.maximum(w3, torch.tensor(0)), l3u) + torch.matmul(torch.minimum(w3, torch.tensor(0)), l3l)
            ol, ou = ol + b3, ou + b3
            
            true_mask = F.one_hot(y_batch, num_classes=10)
            false_mask = 1 - true_mask
            worst = ou[:,:,0] * false_mask + ol[:,:,0] * true_mask
            
            out = model(x_batch)
            batch_loss = kappa*fit_loss(out, y_batch) + (1-kappa)*spec_loss(worst, y_batch)
            batch_loss.backward()
            opt.step()

        tot_test, tot_acc = 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))

In [9]:
robust_model = nn.Sequential(Net())
robust_model = robust_model.to(device)
robust_model.train()
robust_train_model(robust_model, 30, 0.1, 0.5, 1)

Epoch 1: Accuracy 0.93310 [16.86 seconds]
Epoch 2: Accuracy 0.93760 [16.81 seconds]
Epoch 3: Accuracy 0.94010 [17.14 seconds]
Epoch 4: Accuracy 0.94620 [16.86 seconds]
Epoch 5: Accuracy 0.94880 [16.78 seconds]
Epoch 6: Accuracy 0.94950 [17.90 seconds]
Epoch 7: Accuracy 0.95240 [17.23 seconds]
Epoch 8: Accuracy 0.95080 [15.78 seconds]
Epoch 9: Accuracy 0.95350 [17.43 seconds]
Epoch 10: Accuracy 0.95150 [16.28 seconds]
Epoch 11: Accuracy 0.95260 [16.97 seconds]
Epoch 12: Accuracy 0.95000 [17.07 seconds]
Epoch 13: Accuracy 0.94880 [16.45 seconds]
Epoch 14: Accuracy 0.94980 [17.15 seconds]
Epoch 15: Accuracy 0.95010 [19.86 seconds]
Epoch 16: Accuracy 0.94820 [17.53 seconds]
Epoch 17: Accuracy 0.94840 [17.82 seconds]
Epoch 18: Accuracy 0.94840 [17.63 seconds]
Epoch 19: Accuracy 0.94850 [16.95 seconds]
Epoch 20: Accuracy 0.94700 [16.92 seconds]
Epoch 21: Accuracy 0.94800 [17.41 seconds]
Epoch 22: Accuracy 0.94650 [16.91 seconds]
Epoch 23: Accuracy 0.94610 [18.71 seconds]
Epoch 24: Accuracy 0

In [10]:
for i in range(1,11):
    get_robust_accuracy(robust_model, 0.01*i)

Epsilon 0.010: Robust accuracy 0.92550
Epsilon 0.020: Robust accuracy 0.90800
Epsilon 0.030: Robust accuracy 0.89480
Epsilon 0.040: Robust accuracy 0.88300
Epsilon 0.050: Robust accuracy 0.87040
Epsilon 0.060: Robust accuracy 0.85680
Epsilon 0.070: Robust accuracy 0.83660
Epsilon 0.080: Robust accuracy 0.81700
Epsilon 0.090: Robust accuracy 0.79390
Epsilon 0.100: Robust accuracy 0.76750
